In [ ]:
## DONE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install pytypo

In [ ]:
pip install emoji

In [ ]:
import pandas as pd
import numpy as np
import pytypo
import re
import tensorflow as tf
import tensorflow_addons as tfa
import emoji

AUTO = tf.data.experimental.AUTOTUNE

data1 = pd.read_excel("/content/drive/My Drive/DimasASu/Data Latih BDC.xlsx")
data2 = pd.read_csv("/content/drive/My Drive/DimasASu/data_clean.csv")

dataTest = pd.read_csv("/content/drive/My Drive/DimasASu/datatest_labelled.csv")

data1.head() , data2.head()

In [ ]:
print(f"Columns1 {data1.columns}\nColumns2 {data2.columns}\n\nShape1{data1.shape }\nShape2{data2.shape} ")

In [ ]:
data1 = data1[["judul","narasi","label"]]
data2 = data2[["judul","narasi","label"]]

df_train = pd.concat([data1,data2])
df_train = df_train.reset_index().drop("index",axis = 1)
df_train.tail()

In [ ]:
print(df_train.isna().sum())
df_train.dropna(inplace = True)
df_train.isna().sum() , df_train.shape

In [ ]:
sameIndex = []
for key1,val in df_train["judul"].items():
  i = 0
  for key2,val in dataTest["judul"].items():
    if(key1 == key2):
      sameIndex.append(i)
    else:
      i = i + 1
len(sameIndex)

df_train = df_train.drop(sameIndex)
df_train.shape

In [ ]:
df_train = df_train.sample(frac=1,random_state = 0)
df_train.head()

In [ ]:
def clean(text) : 

  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[#]+|[^A-Za-z0-9]+"
  text_cleaning_hash = "#[A-Za-z0-9]+" 
  text_cleaning_num = "(^|\W)\d+"

  text = re.sub(text_cleaning_hash, " ", text).strip()
  text = re.sub(text_cleaning_num, " ", text).strip()
  text = re.sub(text_cleaning_re, " ", text).strip()
  
  text = text.strip()

  out = []
  for word in text.split() :
    # try : 
    #   out.append(word.replace(word, slang[word]))
    # except Exception as e : 
    out.append(word)
      
  return pytypo.correct_sentence(" ".join(out).strip())


In [ ]:
def freeEmoji(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [ ]:
df_train["narasi"] = df_train["narasi"].apply(clean)
df_train["judul"] = df_train["judul"].apply(clean)

dataTest["narasi"] = dataTest["narasi"].apply(clean)
dataTest["judul"] = dataTest["judul"].apply(clean)

df_train["narasi"] = df_train["narasi"].apply(freeEmoji)
df_train["judul"] = df_train["judul"].apply(freeEmoji)

dataTest["narasi"] = dataTest["narasi"].apply(freeEmoji)
dataTest["judul"] = dataTest["judul"].apply(freeEmoji)



In [ ]:
df_train.head(10)

In [ ]:
from transformers import AutoTokenizer,TFAutoModel,AutoModel
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
IndoBert = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p2")

In [ ]:
from collections import Counter
seq_len = [len(i.split()) for i in df_train["narasi"]]

print(Counter(seq_len))

In [ ]:
"""from sklearn.utils import resample
no_claim = df_train[df_train.label == 1]
claim = df_train[df_train.label == 0]
claim.head()
sampled = resample(claim,replace = True,n_samples = len(no_claim),random_state = 0)
df_train = pd.concat([no_claim,sampled])
df_train.shape
sampled"""

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df_train,test_size = 0.05 ,random_state = 93)
df_train.shape , df_val.shape


In [ ]:
tokens_train = {
   "judul":tokenizer.batch_encode_plus(
        df_train.judul.tolist(),
        max_length = 30,
        pad_to_max_length = True,
        truncation = True
   ),
      "narasi":tokenizer.batch_encode_plus(
        df_train.narasi.tolist(),
        max_length = 64,
        pad_to_max_length = True,
        truncation = True
   )
}

tokens_val = {
  'judul' : tokenizer.batch_encode_plus(
    df_val["judul"].tolist(),
    max_length = 30,
    pad_to_max_length=True,
    truncation=True
  ),
  'narasi' : tokenizer.batch_encode_plus(
    df_val["narasi"].tolist(),
    max_length = 64,
    pad_to_max_length=True,
    truncation=True
  )
}

In [ ]:
train_sequence = {
    "judul":tf.convert_to_tensor(tokens_train["judul"]["input_ids"]),
    "narasi":tf.convert_to_tensor(tokens_train["narasi"]["input_ids"])
}
train_mask = {
    "judul":tf.convert_to_tensor(tokens_train["judul"]["attention_mask"]),
    "narasi":tf.convert_to_tensor(tokens_train["narasi"]["attention_mask"])
}
y_train = tf.convert_to_tensor(df_train.label.tolist())

validation_sequence = {
    "judul":tf.convert_to_tensor(tokens_val["judul"]["input_ids"]),
    "narasi":tf.convert_to_tensor(tokens_val["narasi"]["input_ids"])
}
validation_mask = {
    "judul":tf.convert_to_tensor(tokens_val["judul"]["attention_mask"]),
    "narasi":tf.convert_to_tensor(tokens_val["narasi"]["attention_mask"])
}
y_val = tf.convert_to_tensor(df_val.label.tolist())

y_train



In [ ]:
train_data = {
    "judul":tf.squeeze(tf.convert_to_tensor(tokens_train["judul"]["input_ids"])),
    "narasi":tf.squeeze(tf.convert_to_tensor(tokens_train["narasi"]["input_ids"])),
    "judulmask":tf.squeeze(tf.convert_to_tensor(tokens_train["judul"]["attention_mask"])),
    "narasimask":tf.squeeze(tf.convert_to_tensor(tokens_train["narasi"]["attention_mask"]))
}

validation_sequence = {
    "judul":tf.convert_to_tensor(tokens_val["judul"]["input_ids"]),
    "narasi":tf.convert_to_tensor(tokens_val["narasi"]["input_ids"]),
    "judulmask":tf.convert_to_tensor(tokens_val["judul"]["attention_mask"]),
    "narasimask":tf.convert_to_tensor(tokens_val["narasi"]["attention_mask"])
}

y_val = tf.convert_to_tensor(df_val.label.tolist())



In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train,2)
y_val = to_categorical(y_val,2)

In [ ]:
dataset = (tf.data.Dataset.from_tensor_slices((train_data,y_train))
    .batch(32)
    .prefetch(AUTO)
           )
dataVal = (tf.data.Dataset.from_tensor_slices((validation_sequence,y_val))
    .batch(32)
    .prefetch(AUTO)
    .cache()
           )

In [ ]:
Counter(df_train["label"])
Yatoiba = np.log(5592/5680)
Yatoiba

In [ ]:
def makemodel(output_bias = None):


  ids1 = tf.keras.layers.Input(shape=(30,),dtype=tf.int32,name="judul")
  att1 = tf.keras.layers.Input((30,),dtype = tf.int32,name="judulatt")
  output_bias = tf.keras.initializers.Constant(output_bias)
  ids2 = tf.keras.layers.Input((64,),dtype=tf.int32,name="narasi")
  att2 = tf.keras.layers.Input((64,),dtype=tf.int32,name="narasiatt")
    
  embed1,ya = IndoBert(ids1,attention_mask=att1)
  embed2,toi= IndoBert(ids2,attention_mask=att2)

  embed3,iba = IndoBert(ids1,attention_mask=att1)
  embed4,yatoiba = IndoBert(ids2,attention_mask=att2)

  y = tf.keras.layers.concatenate([iba,yatoiba])
  y = tf.keras.layers.Masking(mask_value = 0)(y)
  y = tf.keras.layers.Dense(units = 32,activation = "relu")(y)

  x = tf.keras.layers.concatenate([ya,toi])
  x = tf.keras.layers.Masking(mask_value = 0)(x)
  x = tf.keras.layers.Dense(16,activation = 'relu')(x)

  fin =  tf.keras.layers.concatenate([x,y])

  fin2 = tf.keras.layers.GlobalAveragePooling1D()(embed1)
  fin3 = tf.keras.layers.GlobalAveragePooling1D()(embed2)
  fin4 = tf.keras.layers.GlobalAveragePooling1D()(embed3)
  fin4 = tf.keras.layers.Dense(units = 64, activation="relu")(fin4)

  fin5 = tf.keras.layers.GlobalAveragePooling1D()(embed4)
  fin5 = tf.keras.layers.Dense(units = 32, activation = "relu")(fin5)

  fin = tf.keras.layers.concatenate([fin,fin2,fin3,fin4,fin5])
  fin = tf.keras.layers.Dropout(0.35)(fin)
  fin = tf.keras.layers.Dense(units = 34,activation = "relu")(fin)
  fin = tf.keras.layers.Dropout(0.3)(fin)
  fin = tf.keras.layers.Dense(2,activation="softmax")(fin)




  final = tf.keras.Model(inputs=[ids1, att1,ids2,att2],outputs=[fin])
  METRICS = [
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
  ] 


  for layer in final.layers:
    layer.trainable = False
  for layer in final.layers[-7:]:
      layer.trainable = True

  optimizer = tf.keras.optimizers.Adam(learning_rate=.001,clipnorm=0.001)
  final.compile(loss=tfa.losses.SigmoidFocalCrossEntropy(), optimizer=optimizer,metrics = METRICS)

  return final

final = makemodel(Yatoiba)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(final,"modelarhcitect.png",show_shapes=True)

In [ ]:
final.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
EarlyStop = EarlyStopping(monitor="val_loss",patience = 5)

In [ ]:
import math
def create_learning_rate_scheduler(max_learn_rate=1e-2,
                                   end_learn_rate=1e-6,
                                   warmup_epoch_count=3,
                                   total_epoch_count=15):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = max_learn_rate
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

In [ ]:
history3=final.fit(x = dataset,epochs = 15,validation_data=dataVal,callbacks = [EarlyStop, create_learning_rate_scheduler(max_learn_rate=1e-3,
                                                    end_learn_rate=1e-6,
                                                    warmup_epoch_count=3,
                                                    total_epoch_count=15),EarlyStop])

In [ ]:
test_seq = {
  'judul' : tokenizer.batch_encode_plus(
    dataTest["judul"].tolist(),
    max_length = 30,
    pad_to_max_length=True,
    truncation=True
  ),
  'narasi' : tokenizer.batch_encode_plus(
    dataTest["narasi"].tolist(),
    max_length = 64,
    pad_to_max_length=True,
    truncation=True
  )
}

test_sequence = {
    "judul":tf.convert_to_tensor(test_seq["judul"]["input_ids"]),
    "narasi":tf.convert_to_tensor(test_seq["narasi"]["input_ids"]),
    "judulmask":tf.convert_to_tensor(test_seq["judul"]["attention_mask"]),
    "narasimask":tf.convert_to_tensor(test_seq["narasi"]["attention_mask"])
}



In [ ]:
y_test = dataTest["label"]

In [ ]:
y_pred = np.argmax(final.predict(test_sequence),axis = 1)
y_pred[:5]

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

f1_score(y_true = y_test , y_pred = y_pred,average='micro')